In [ ]:
!nvidia-smi

Sun Mar 27 13:44:15 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 12148, done.
remote: Total 12148 (delta 0), reused 0 (delta 0), pack-reused 12148
Receiving objects: 100% (12148/12148), 11.42 MiB | 17.33 MiB/s, done.
Resolving deltas: 100% (8441/8441), done.


In [ ]:
!pip install -qU wandb
!pip install -qU bbox-utility

     |████████████████████████████████| 1.7 MB 15.5 MB/s 
     |████████████████████████████████| 144 kB 79.1 MB/s 
     |████████████████████████████████| 181 kB 87.3 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 58 kB 5.3 MB/s 
     |████████████████████████████████| 3.3 MB 22.8 MB/s 
     |████████████████████████████████| 34.5 MB 117.4 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip '/content/drive/MyDrive/input/happy-whale-and-dolphin.zip' -d '/content/data/'

In [ ]:
!unzip '/content/drive/MyDrive/input/detection_labels.zip' -d /content/labels_txt

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: /content/labels_txt/d292d874624575_jpg.rf.f2730ac32fcd86dc81b6e4f3eece6539.txt  
  inflating: /content/labels_txt/d293ed90e51c6c_jpg.rf.33d2eb1f65b48f405685cbca55184d4a.txt  
  inflating: /content/labels_txt/d2949b7b6b8f85_jpg.rf.5afa63574d74b41c5c6ebb3700a5ad85.txt  
  inflating: /content/labels_txt/d2951efcf367f2_jpg.rf.e7ef88063ed3f3eebbf70dd83ee3af9d.txt  
  inflating: /content/labels_txt/d2984e84804752_jpg.rf.d3b3b301672cff7ca3c6752c06bcfbbf.txt  
  inflating: /content/labels_txt/d29921fa541a64_jpg.rf.3d8483eee1eb7e60878f804c4e04b9aa.txt  
  inflating: /content/labels_txt/d29cc494f2aa55_jpg.rf.9bcf902859846d4393f6b6632783fd30.txt  
  inflating: /content/labels_txt/d29d89d30c5007_jpg.rf.4390ef1a5bd6ce46cf9e8825201e8fb3.txt  
  inflating: /content/labels_txt/d29e9134a6414a_jpg.rf.390b83c0790556d3cd0d3fcadc425733.txt  
  inflating: /content/labels_txt/d2a7440e392d88_jpg.rf.442a4b83ad73466d328a03e466cd870f.txt  
  inflating: /content/la

In [ ]:
import numpy as np
from tqdm.notebook import tqdm
tqdm.pandas()
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import glob

import shutil
import sys
sys.path.append('../input/tensorflow-great-barrier-reef')

from joblib import Parallel, delayed

from IPython.display import display, HTML

from matplotlib import animation, rc
rc('animation', html='jshtml')

In [ ]:
import wandb

try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    api_key = user_secrets.get_secret("WANDB")
    wandb.login(key=api_key)
    anonymous = None
except:
    wandb.login(anonymous='must')
    print('To use your W&B account,\nGo to Add-ons -> Secrets and provide your W&B access token. Use the Label name as WANDB. \nGet your W&B access token from here: https://wandb.ai/authorize')

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


To use your W&B account,
Go to Add-ons -> Secrets and provide your W&B access token. Use the Label name as WANDB. 
Get your W&B access token from here: https://wandb.ai/authorize


In [ ]:
FOLD      = 1 # which fold to train
DIM       = 1024*2
MODEL     = 'yolov5m6'
BATCH     = 4
EPOCHS    = 100

PROJECT   = 'whale' # w&b in yolov5
NAME      = f'{MODEL}-dim{DIM}-fold{FOLD}' # w&b for yolov5

REMOVE_NOBBOX = True # remove images with no bbox
ROOT_DIR  = '/content/data'
IMAGE_DIR = '/content/images' # directory to save images
LABEL_DIR = '/content/labels_' # directory to save labels

In [ ]:
# !rm -rf /content/images
!rm -rf /content/labels

In [ ]:
!mkdir -p {IMAGE_DIR}
!mkdir -p {LABEL_DIR}

In [ ]:
os.makedirs('/content/images/train', exist_ok=True)
os.makedirs('/content/images/val', exist_ok=True)
os.makedirs('/content/labels/train', exist_ok=True)
os.makedirs('/content/labels/val', exist_ok=True)

In [ ]:
for i in sorted(glob('/content/data/*.txt')):
    if 'README' in i:
        print(i)

/content/data/README.dataset.txt
/content/data/README.roboflow.txt


In [ ]:
from glob import glob
from shutil import copyfile

def make_dataset(mode='train'):
    txt_paths = sorted(glob('/content/labels_txt/*.txt'))
    if mode == 'train':
        pgb = txt_paths[:int(len(txt_paths)*1.0)]
    else:
        pgb = txt_paths[int(len(txt_paths)*0.95):]
    for i in pgb:
        if 'README' in i:
            continue
        file_name = i.split('/')[-1].split('.')[0].split('_')[0]
        copyfile('/content/data/test_images/'+file_name+'.jpg', f'/content/images/{mode}/' + file_name + '.jpg')
        f = open(i, 'r')
        k = open(f'/content/labels/{mode}/'+file_name+'.txt', 'w')
        for j in f.readlines():
            k.write(j)

        f.close()
        k.close()
        # break

In [ ]:
make_dataset(mode='train')
make_dataset(mode='val')

In [ ]:
import yaml

cwd = '/content/'

data = dict(
    path  = '/content',
    train =  '/content/images/train' ,
    val   =  '/content/images/val',
    nc    = 1,
    names = ['fin'],
    )

with open(os.path.join( cwd , 'gbr.yaml'), 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)

f = open(os.path.join( cwd , 'gbr.yaml'), 'r')
print('\nyaml:')
print(f.read())


yaml:
names:
- fin
nc: 1
path: /content
train: /content/images/train
val: /content/images/val



In [ ]:
%%writefile /content/hyp.yaml
lr0: 0.01  # initial learning rate (SGD=1E-2, Adam=1E-3)
lrf: 0.1  # final OneCycleLR learning rate (lr0 * lrf)
momentum: 0.937  # SGD momentum/Adam beta1
weight_decay: 0.0005  # optimizer weight decay 5e-4
warmup_epochs: 3.0  # warmup epochs (fractions ok)
warmup_momentum: 0.8  # warmup initial momentum
warmup_bias_lr: 0.1  # warmup initial bias lr
box: 0.05  # box loss gain
cls: 0.5  # cls loss gain
cls_pw: 1.0  # cls BCELoss positive_weight
obj: 1.0  # obj loss gain (scale with pixels)
obj_pw: 1.0  # obj BCELoss positive_weight
iou_t: 0.20  # IoU training threshold
anchor_t: 4.0  # anchor-multiple threshold
# anchors: 3  # anchors per output layer (0 to ignore)
fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5)
hsv_h: 0.015  # image HSV-Hue augmentation (fraction)
hsv_s: 0.7  # image HSV-Saturation augmentation (fraction)
hsv_v: 0.4  # image HSV-Value augmentation (fraction)
degrees: 0.1 # 0.30  # image rotation (+/- deg)
translate: 0.10  # image translation (+/- fraction)
scale: 0.10  # image scale (+/- gain)
shear: 0.0 # 2.0  # image shear (+/- deg)
perspective: 0.0  # image perspective (+/- fraction), range 0-0.001
flipud: 0.0  # image flip up-down (probability)
fliplr: 0.5  # image flip left-right (probability)
mosaic: 0.0 # 0.2  # image mosaic (probability)
mixup: 0.0 # 0.5 # image mixup (probability)
copy_paste: 0.0  # segment copy-paste (probability)

Writing /content/hyp.yaml


In [ ]:
%cd /content
%cd yolov5
%pip install -qr requirements.txt  # install

from yolov5 import utils
display = utils.notebook_init()  # check

YOLOv5 🚀 v6.1-69-g7830e91 torch 1.10.0+cu111 CUDA:0 (Tesla V100-SXM2-16GB, 16160MiB)


Setup complete ✅ (8 CPUs, 51.0 GB RAM, 120.8/166.8 GB disk)


In [ ]:
!python train.py --img {DIM} \
--batch {BATCH} \
--epochs {EPOCHS} \
--data /content/gbr.yaml \
--hyp /content/hyp.yaml \
--weights {MODEL}.pt \
--project {PROJECT} --name {NAME} \
--resume /content/yolov5/whale/yolov5m6-dim2048-fold1/weights/last.pt \
--save-period 1 \
--exist-ok \
--single-cls

wandb: Currently logged in as: jcdata (use `wandb login --relogin` to force relogin)
train: weights=yolov5m6.pt, cfg=, data=/content/gbr.yaml, hyp=/content/hyp.yaml, epochs=100, batch_size=4, imgsz=2048, rect=False, resume=/content/yolov5/whale/yolov5m6-dim2048-fold1/weights/last.pt, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=True, optimizer=SGD, sync_bn=False, workers=8, project=whale, name=yolov5m6-dim2048-fold1, exist_ok=True, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
Resuming training from /content/yolov5/whale/yolov5m6-dim2048-fold1/weights/last.pt
YOLOv5 🚀 v6.1-69-g7830e91 torch 1.10.0+cu111 CUDA:0 (Tesla V100-SXM2-16GB, 16160MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weig

In [ ]:
!python train.py --img {DIM} \
--batch {BATCH} \
--epochs {EPOCHS} \
--data /content/gbr.yaml \
--hyp /content/hyp.yaml \
--weights {MODEL}.pt \
--project {PROJECT} --name {NAME} \
--resume /content/yolov5/whale/yolov5m6-dim2048-fold1/weights/last_epoch18 \
--save-period 1 \
--single-cls

In [ ]:
--resume true \
# --weights {MODEL}.pt \
--save-period 1 \